In [1]:
from typing import Dict
from sinapses_cliente.modelo.requisicao.requisicao import Requisicao
from sinapses_cliente.modelo.requisicao.mensagem import Mensagem

class RequisicaoAILA(Requisicao):
    
    def __init__(self, requisicao: Dict[str,any]):

        if 'mensagem' not in requisicao:
            raise Exception('A mensagem não foi definida!')

        self.mensagem = Mensagem.from_dict(requisicao['mensagem'])

        super().__init__(requisicao)

In [2]:
from typing import Callable

from sinapses_cliente.modelo.resposta.resposta import Resposta

from sinapses_cliente.modelo.treinamento_resultado import TreinamentoResultado
from sinapses_cliente.modelo.servico_contexto import ServicoContexto
from sinapses_cliente.sinapses_pipeline import SinapsesPipeline
from sinapses_cliente.sinapses_sessao import SinapsesSessao

In [3]:
pipeline = SinapsesPipeline.instancia();

O arquivo de configuração foi encontrado em: /home/sinapses/curso/projetos/aila/sinapses.yml
Validando as configurações do arquivo encontrado...
 - O endereço do sinapses é válido!
 - A autenticação é válida!
 - O id da versão do modelo é válida!
Arquivo de configuração válido!
Instanciando o cliente do Sinapses - Versão: 1.4.0


In [4]:
def funcao_treinamento(sessao: SinapsesSessao) -> TreinamentoResultado:
    return TreinamentoResultado(1, [])

In [5]:
import os
import sys
import pickle
import zipfile
from io import BytesIO
from pathlib import Path

def save_file(modelo, filename):
    f = open(filename, "wb")
    f.write(get_file(modelo, filename))
    f.close()
    
def get_file(modelo, filename):
    buffer = modelo.get_recurso_por_nome(filename).conteudo.bytes
    return buffer

def unzip(modelo, filename):
    z = zipfile.ZipFile(BytesIO(get_file(modelo, filename)))
    z.extractall('./')
    z.close()

In [6]:
def funcao_inicializacao_servico(sessao: SinapsesSessao):
    print('iniciando servico AILA UNIFOR')
    modelo = sessao.get_modelo_versao()
    
    save_file(modelo, 'main.py')

    print('servico AILA UNIFOR iniciado com sucesso')
    
    return ServicoContexto(
    )

In [7]:
def funcao_servico(sessao: SinapsesSessao, ctx: ServicoContexto, requisicao: RequisicaoAILA) -> Resposta:
    texto = sessao.get_mensagem_texto(requisicao.mensagem)

    from main import AILA

    resposta = None
    try:
        resultado = AILA(texto)
        resposta = Resposta(resultado)
        return resposta
    except Exception as e: 
        return Resposta(str(e))

In [8]:
def funcao_teste_servico(_s: SinapsesSessao, _c: ServicoContexto, servico: Callable[[str], str]) -> None:    
    texto = 'Art. 1.000 do Código Civil'
    
    r1 = servico({'mensagem': Mensagem.texto(texto).to_dict()})
    print({'mensagem': Mensagem.texto(texto).to_dict()})
    print(r1)
    assert True == True

In [9]:
pipeline.definir_treinamento(funcao_treinamento)
pipeline.definir_inicializacao_servico(funcao_inicializacao_servico)
pipeline.definir_servico(funcao_servico)
pipeline.definir_teste_servico(funcao_teste_servico)

In [10]:
# pipeline.testar_pipeline()